In [6]:
!pip install --upgrade gensim -q
!pip install matplotlib -q
!pip install spacy
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
wv = api.load('glove-wiki-gigaword-50')

In [8]:
import csv
gender_specific = ['he','him','man','his','boy','uncle','father','grandfather',
                   'she','her','woman','girl','aunt','mother','grandmother']
matrix = []
for word in gender_specific:
    lis = wv[word]
    matrix.append(lis)

In [9]:
import pandas as pd
from sklearn.preprocessing import normalize
def gender_subspace(gender_specific,matrix):
    matrix = np.array(matrix)
    gender_subspace = np.mean(matrix, axis=0)
    return gender_specific,gender_subspace

In [10]:
def hard_debias(dict, words, space):
    lis = np.array(dict.index_to_key)
    vect = np.array((dict.vectors))
    vect_norm = normalize(vect)
    gender_subspace_norm = normalize(space.reshape(1,-1))
    
    gender_project = np.dot(vect_norm, gender_subspace_norm.T)*gender_subspace_norm
    bias_subspace = np.sum(gender_project, axis=0)
    
    bias_norm = normalize(bias_subspace.reshape(1,-1))
    ortho_complement = np.eye(bias_norm.shape[1]) - bias_norm.T*bias_norm
    gender_subspace_debias = np.dot(gender_subspace_norm, ortho_complement).reshape((50,))
    print(np.shape(gender_subspace_debias))
    # debiased_dict[word] = vect - np.dot(vect, gender_subspace_debias)*gender_subspace_debias.reshape(-1,50)
    print("HUIE")
    print("HUIE")
    debiased_dict = {}
    print("HUIE")
    for i,word in enumerate(lis):
        debiased_dict[word] = (vect[i] - (np.dot(vect[i], gender_subspace_debias))*gender_subspace_debias).tolist()
    return debiased_dict

In [11]:
words, space = gender_subspace(gender_specific,matrix)
new_dict = hard_debias(wv, words, space)

(50,)
HUIE
HUIE
HUIE


In [50]:
print(wv['apple'])

[ 0.52042  -0.8314    0.49961   1.2893    0.1151    0.057521 -1.3753
 -0.97313   0.18346   0.47672  -0.15112   0.35532   0.25912  -0.77857
  0.52181   0.47695  -1.4251    0.858     0.59821  -1.0903    0.33574
 -0.60891   0.41742   0.21569  -0.07417  -0.5822   -0.4502    0.17253
  0.16448  -0.38413   2.3283   -0.66682  -0.58181   0.74389   0.095015
 -0.47865  -0.84591   0.38704   0.23693  -1.5523    0.64802  -0.16521
 -1.4719   -0.16224   0.79857   0.97391   0.40027  -0.21912  -0.30938
  0.26581 ]


In [49]:
print(new_dict['apple'])

[ 0.52042001 -0.83139998  0.49961001  1.28929996  0.1151      0.057521
 -1.37530005 -0.97312999  0.18346     0.47672001 -0.15112001  0.35532001
  0.25911999 -0.77857     0.52181     0.47694999 -1.42509997  0.85799998
  0.59820998 -1.09029996  0.33574    -0.60891002  0.41742     0.21569
 -0.07417    -0.58219999 -0.45019999  0.17253     0.16448    -0.38413
  2.3283     -0.66681999 -0.58181     0.74388999  0.095015   -0.47865
 -0.84591001  0.38703999  0.23693    -1.55229998  0.64802003 -0.16520999
 -1.47189999 -0.16224     0.79856998  0.97390997  0.40026999 -0.21912
 -0.30937999  0.26581001]


In [12]:
import json
json_o = json.dumps(new_dict)
jsonFile = open("data.json", "w")
jsonFile.write(json_o)
jsonFile.close()